# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 3:26PM,240895,10,SO94309,"Senseonics, Incorporated",Released
1,Jun 30 2022 3:25PM,240894,20,SSNA-8214262,Sartorius Stedim North America,Released
2,Jun 30 2022 3:23PM,240893,19,N17655,"NAPP Technologies, LLC",Released
3,Jun 30 2022 3:20PM,240892,16,AIC-378145,American International Chemical,Released
4,Jun 30 2022 3:10PM,240891,19,60016181.2,"GUSA Granules USA, Inc.",Released
5,Jun 30 2022 3:10PM,240890,10,0085817469,ISDIN Corporation,Released
6,Jun 30 2022 3:10PM,240888,10,0085817473,ISDIN Corporation,Released
7,Jun 30 2022 3:10PM,240888,10,0085817400,ISDIN Corporation,Released
8,Jun 30 2022 3:10PM,240888,10,0085817462,ISDIN Corporation,Released
9,Jun 30 2022 3:10PM,240888,10,0085817460,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,240891,Released,1
34,240892,Released,1
35,240893,Released,1
36,240894,Released,1
37,240895,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240891,NaN,NaN,1.0
240892,NaN,NaN,1.0
240893,NaN,NaN,1.0
240894,NaN,NaN,1.0
240895,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240784,0.0,0.0,1.0
240786,0.0,0.0,1.0
240800,24.0,30.0,0.0
240804,0.0,2.0,1.0
240805,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240784,0,0,1
240786,0,0,1
240800,24,30,0
240804,0,2,1
240805,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240784,0,0,1
1,240786,0,0,1
2,240800,24,30,0
3,240804,0,2,1
4,240805,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240784,,,1
1,240786,,,1
2,240800,24,30,
3,240804,,2,1
4,240805,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 3:26PM,240895,10,"Senseonics, Incorporated"
1,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America
2,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC"
3,Jun 30 2022 3:20PM,240892,16,American International Chemical
4,Jun 30 2022 3:10PM,240891,19,"GUSA Granules USA, Inc."
5,Jun 30 2022 3:10PM,240890,10,ISDIN Corporation
6,Jun 30 2022 3:10PM,240888,10,ISDIN Corporation
24,Jun 30 2022 3:08PM,240889,10,Yusen – 3D Matrix
25,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation
65,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 3:26PM,240895,10,"Senseonics, Incorporated",,,1
1,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,,,1
2,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",,,1
3,Jun 30 2022 3:20PM,240892,16,American International Chemical,,,1
4,Jun 30 2022 3:10PM,240891,19,"GUSA Granules USA, Inc.",,,1
5,Jun 30 2022 3:10PM,240890,10,ISDIN Corporation,,,1
6,Jun 30 2022 3:10PM,240888,10,ISDIN Corporation,,,18
7,Jun 30 2022 3:08PM,240889,10,Yusen – 3D Matrix,,,1
8,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,,,40
9,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:26PM,240895,10,"Senseonics, Incorporated",1,,
1,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,1,,
2,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",1,,
3,Jun 30 2022 3:20PM,240892,16,American International Chemical,1,,
4,Jun 30 2022 3:10PM,240891,19,"GUSA Granules USA, Inc.",1,,
5,Jun 30 2022 3:10PM,240890,10,ISDIN Corporation,1,,
6,Jun 30 2022 3:10PM,240888,10,ISDIN Corporation,18,,
7,Jun 30 2022 3:08PM,240889,10,Yusen – 3D Matrix,1,,
8,Jun 30 2022 3:02PM,240884,10,ISDIN Corporation,40,,
9,Jun 30 2022 3:01PM,240887,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:26PM,240895,10,"Senseonics, Incorporated",1,,
1,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,1,,
2,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",1,,
3,Jun 30 2022 3:20PM,240892,16,American International Chemical,1,,
4,Jun 30 2022 3:10PM,240891,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:26PM,240895,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,1.0,NaN,NaN
2,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",1.0,NaN,NaN
3,Jun 30 2022 3:20PM,240892,16,American International Chemical,1.0,NaN,NaN
4,Jun 30 2022 3:10PM,240891,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 3:26PM,240895,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,1.0,0.0,0.0
2,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",1.0,0.0,0.0
3,Jun 30 2022 3:20PM,240892,16,American International Chemical,1.0,0.0,0.0
4,Jun 30 2022 3:10PM,240891,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3131437,89.0,0.0,0.0
12,481756,8.0,0.0,0.0
15,240847,0.0,29.0,0.0
16,1204249,3.0,2.0,0.0
19,1685973,6.0,32.0,24.0
20,963449,23.0,9.0,0.0
21,481570,2.0,0.0,0.0
22,240846,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3131437,89.0,0.0,0.0
1,12,481756,8.0,0.0,0.0
2,15,240847,0.0,29.0,0.0
3,16,1204249,3.0,2.0,0.0
4,19,1685973,6.0,32.0,24.0
5,20,963449,23.0,9.0,0.0
6,21,481570,2.0,0.0,0.0
7,22,240846,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,89.0,0.0,0.0
1,12,8.0,0.0,0.0
2,15,0.0,29.0,0.0
3,16,3.0,2.0,0.0
4,19,6.0,32.0,24.0
5,20,23.0,9.0,0.0
6,21,2.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,89.0
1,12,Released,8.0
2,15,Released,0.0
3,16,Released,3.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
Executing,0.0,0.0,29.0,2.0,32.0,9.0,0.0,0.0
Released,89.0,8.0,0.0,3.0,6.0,23.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,Executing,0.0,0.0,29.0,2.0,32.0,9.0,0.0,0.0
2,Released,89.0,8.0,0.0,3.0,6.0,23.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,Executing,0.0,0.0,29.0,2.0,32.0,9.0,0.0,0.0
2,Released,89.0,8.0,0.0,3.0,6.0,23.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()